# Pre processing of StackOverflow data

## Import libraries

In [1]:
import pandas as pd
import re, string
import networkx as nx
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import pandas as pd
import numpy as np 
import re
import requests
from random import choice
import networkx as nx
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rasmusblirupjensen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load data

In [2]:
postsA = pd.read_csv("/Users/rasmusblirupjensen/Downloads/postsA.csv", sep=",", index_col=0)
postsQ = pd.read_csv("/Users/rasmusblirupjensen/Downloads/postsQ.csv", sep=",", index_col=0)
users = pd.read_csv("/Users/rasmusblirupjensen/Downloads/complete_users.csv", sep=",", index_col=0)
# Getting list of iso countries
iso_countries = pd.read_csv("https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv")
#Getting list of states in US
states = pd.read_csv("https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/master/notebooks/tutorials/bigquery/resources/us-states.csv", sep=",")
#users = pd.read_csv("data/raw/complete_users.csv", sep=",", index_col=0)
#Only taking questions that has been viewed more than 999 times
postsQ = postsQ[postsQ["view_count"] >=  1000]
#Getting the sentiment list
sent = pd.read_csv("Data_Set_S1.txt", sep="\t")

/Users/rasmusblirupjensen/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Minimizing datasets

A network is made, this is done on order to get a network, and then work reverse, and from here determine which users to continue working with. This is to limit the size of the dataset aswell. 

In [3]:
#store the network
G = nx.DiGraph()
for ind in postsQ.index:
    for i in postsA[postsA['parent_id'] == postsQ["id"][ind]].index:
        if len(postsA[postsA['parent_id'] == postsQ["id"][ind]].index) > 1:
            G.add_edge(postsQ["owner_user_id"][ind],postsA["owner_user_id"][i])
        else: 
            pass
#Removing Isolates
for i in list(nx.isolates(G)):
    G.remove_node(i)
# making graph undirected
G_1 = G.to_undirected()
#getting the largest connected network
largest_cc = sorted(nx.connected_components(G_1),key=len, reverse=True)
#creating the gaint network from the original network
G_2 = G_1.subgraph(largest_cc[0])
 # making a list of users in the network
users_to_use = list(G_2.nodes())



### Filtering dataset to only contain users in the network

In [4]:
postsQ = postsQ[postsQ["owner_user_id"].isin(users_to_use)]
parent_id = list(postsQ["id"].unique())
postsA = postsA[postsA["owner_user_id"].isin(users_to_use)]
postsA = postsA[postsA["parent_id"].isin(parent_id)]
users = users[users["id"].isin(users_to_use)]


In [5]:
print("Number of questions from the users, in the network:",len(postsQ))
print("Number of answers to the all of the questions:",len(postsA))
print("Number of unique users:",len(users))
print()
print("On average each questions has {} responses".format(len(postsA["owner_user_id"])/len(postsQ["owner_user_id"])))

Number of questions from the users, in the network: 6971
Number of answers to the all of the questions: 8459
Number of unique users: 4209

On average each questions has 1.2134557452302395 responses


## Creating Datetime 

In [6]:
postsQ["creation_date"] = pd.to_datetime(postsQ["creation_date"]).dt.tz_localize(None) #turning into datemtime
postsQ["Hour_of_day"] = postsQ["creation_date"].dt.hour #taking only hour from datetime
postsQ["Daily_date"] = postsQ["creation_date"].dt.to_period("D") #Taking only year,month and date from datetime
postsQ["Day_of_week"] = postsQ["creation_date"].dt.day_name() #getting the weekday name 
postsA["creation_date"] = pd.to_datetime(postsA["creation_date"]).dt.tz_localize(None) #turning into datemtime
postsA["Hour_of_day"] = postsA["creation_date"].dt.hour #taking only hour from datetime
postsA["Daily_date"] = postsA["creation_date"].dt.to_period("D") #Taking only year,month and date from datetime
postsA["Day_of_week"] = postsA["creation_date"].dt.day_name()#getting the weekday name 
users["creation_date"] = pd.to_datetime(users["creation_date"]).dt.tz_localize(None) #turning into datemtime
users["last_access_date"] = pd.to_datetime(users["last_access_date"]).dt.tz_localize(None) #turning into datemtime
users["access_hour_of_day"] = users["last_access_date"].dt.hour #taking only hour from datetime
users["Daily_date"] = users["last_access_date"].dt.to_period("D") #Taking only year,month and date from datetime
users["Day_of_week"] = users["last_access_date"].dt.day_name()#getting the weekday name 

---
# Text processing

## Removing html tags and uris from Users, Questions and Answers

In [7]:
import nltk
words = set(nltk.corpus.words.words())

def remove_tags_uris(body):
    li = []
    text = re.findall(r'[a-zA-Z]+', str(body)) # finding all words in the html body
    for i in text:
        if len(i) > 2:
            li.append(i.lower()) # Lowercasing all words
        else:
            pass
    text = " ".join(w for w in li \
         if w.lower() in words or not w.isalpha()) # creating one string with all words
    text = text.replace("code","")
    text = text.replace("quot","")
    if text:
        return text
    else:
        pass

In [8]:
# This could take a while (approx 90 minutes)
postsA["content"] = postsA["body"].apply(lambda row: remove_tags_uris(row))
print("Done Answers")
postsQ["content_title"] = postsQ["title"].apply(lambda row: remove_tags_uris(row))
postsQ["content_body"] = postsQ["body"].apply(lambda row: remove_tags_uris(row))
print("Done Questions")
users["content"] = users["about_me"].apply(lambda row: remove_tags_uris(row))
print("Done Users")

Done Answers
Done Questions
Done Users


In [9]:
postsQ

,id,title,body,accepted_answer_id,answer_count,comment_count,owner_user_id,score,tags,view_count,creation_date,Hour_of_day,Daily_date,Day_of_week,content_title,content_body
96,59684674,Should I add Python's pyc files to .dockerignore?,<p>I've seen several examples of <code>.docker...,59786706,1,3,275088,9,python|docker|compilation|docker-build,1084,2020-01-10 15:34:31.800,15,2020-01-10,Friday,should add python,seen several for python where and are ...
113,59613434,How to produce map with distinct values from a...,"<p>I have a map <code>Map&lt;K, V&gt;</code> a...",59613610,5,8,3764965,13,java|dictionary|java-8|java-stream|collectors,1350,2020-01-06 13:59:06.163,13,2020-01-06,Monday,how produce map with distinct from map and use...,have map map and goal remove the and output ...
168,57583761,how to show error message in react js when htt...,<p>could you please tell me how to show error ...,57583854,3,0,3701974,0,javascript|node.js|reactjs|axios,2017,2019-08-21 02:39:01.900,2,2019-08-21,Wednesday,how show error message react when request send,could you please tell how show error message r...
172,57302715,how to get input field value on button click i...,<p>Could you please tell me how to get input f...,57303641,8,1,3701974,5,javascript|reactjs,3864,2019-08-01 06:06:49.163,6,2019-08-01,Thursday,how get input field value button click react,could you please tell how get input field valu...
183,57242587,Why is the number of local variables used in a...,<p>I have a snippet of simple Java code:</p>\n...,57242675,5,2,243990,30,java|jvm|javac|bytecode,3361,2019-07-28 15:58:50.453,15,2019-07-28,Sunday,why the number local used method not the most ...,have snippet simple public static void main ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320845,52238803,How to convert list of dictionaries into Pyspa...,<p>I want to convert my list of dictionaries i...,52243379,4,0,8952023,10,python|pyspark|apache-spark-sql,15667,2018-09-08 19:41:49.187,19,2018-09-08,Saturday,how convert list into,want convert list into this the list type act...
320860,51994326,jsPDF - include other pdf,<p>I'm trying to solve such problem with jsPDF...,51994639,2,0,7286148,3,javascript|pdf|merge|append|jspdf,2956,2018-08-23 21:10:45.870,21,2018-08-23,Thursday,include other,trying solve such problem with have file which...
320909,52212997,Permission for reading/writing to the root fol...,<p>Are there any permissions I can/must declar...,52213869,2,2,2300597,5,java|android|android-permissions|android-security,1655,2018-09-06 22:44:02.927,22,2018-09-06,Thursday,permission for reading writing the root folder...,are there any can must declare want the able s...
320921,51975512,Faster alternative to perform pandas groupby o...,"<p>I have a dataset with name (person_name), d...",51975680,5,3,10236633,8,python|pandas|numpy|pandas-groupby,6248,2018-08-22 21:38:41.067,21,2018-08-22,Wednesday,faster alternative perform operation,have with name person name day and color shirt...


## Removing punctuatin from Users, Questions and Answers

In [10]:
def remove_punct(x):
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',str(x))
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [11]:
postsA["content"] = postsA["content"].apply(lambda row: remove_punct(row))
postsQ["content_title"] = postsQ["content_title"].apply(lambda row: remove_punct(row))
postsQ["content_body"] = postsQ["body"].apply(lambda row: remove_punct(row))
users["content"] = users["content"].apply(lambda row: remove_punct(row))

## Removing stop words from Users, Questions and Answers

In [12]:
stops = set(stopwords.words("english"))
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

In [13]:
postsA["content"] = postsA["content"].apply(lambda row: removeStopwords(row))
postsQ["content_title"] = postsQ["content_title"].apply(lambda row: removeStopwords(row))
postsQ["content_body"] = postsQ["content_body"].apply(lambda row: removeStopwords(row))
users["content"] = users["content"].apply(lambda row: removeStopwords(row))

## Calculatin Sentiment of answers, about users and questions

In [14]:
#sent = pd.read_csv("Data_Set_S1.txt", sep="\t") # getting the word sentiment data
sent = sent[["word","happiness_average"]] #taking only relevant columns
sent_words = sent["word"].tolist() # creating a list of the words in the sentiment data
sent_dict= dict(zip(sent.word, sent.happiness_average)) #creating a dictionary, where words are keys and the score is the value

In [15]:
def sentiment(x):
    num = []
    text = str(x).split(" ") #splitting the string of words into a list of word
    for i in text:
        if i in sent_words:
            num.append(sent_dict[i])
        else:
            num.append(np.nan)
        return np.mean(num)

# calcuting the average sentiment of the users answers
def answer_sentiment(x, df, column):
    return  df[df["owner_user_id"] == x][column].mean()

In [16]:
postsQ["Sentiment_title"] = postsQ["content_title"].apply(lambda x: sentiment(x))
print("Done")
postsQ["Sentiment_body"] = postsQ["content_body"].apply(lambda x: sentiment(x))
print("Done")
postsA["Sentiment"] = postsA["content"].apply(lambda x: sentiment(x))
print("Done")
users["Sentiment_aboutme"] = users["content"].apply(lambda x: sentiment(x))
print("Done")
users["Sentiment_answer"] = users["id"].apply(lambda x: answer_sentiment(x, postsA, "Sentiment"))
print("Done")
users["Sentiment_question"] = users["id"].apply(lambda x: answer_sentiment(x, postsQ, "Sentiment_body"))
print("Done")


Done
Done
Done
Done
Done
Done


## Splitting tags to a list and extracting language of Questions

In [17]:
# The first element in the list is taken, as this represents the overall language of the question
postsQ["language"] = postsQ["tags"].apply(lambda x: x.split("|")[0])


## Filter Questions to only considering most popular languages

In [18]:
postsQ["language"] = postsQ["language"].apply(lambda row: re.sub(r'^python.*','python', row))
postsQ["language"] = postsQ["language"].apply(lambda row: re.sub(r'^.*sql.*','sql', row))
postsQ= postsQ.drop(columns=["answer_count", "comment_count"], axis=1)
top_language = pd.DataFrame(postsQ["language"].value_counts().sort_values(ascending=False)).head(13).index
postsQ = postsQ[postsQ["language"].isin(top_language)]
postsQ["language"] = postsQ['language'].apply(lambda row: row.replace('javafx','java'))
postsQ["language"] = postsQ['language'].apply(lambda row: row.replace('java-8','java'))
postsQ["language"] = postsQ['language'].apply(lambda row: row.replace('c#','c #'))

## Cleaning location of Users

In [19]:
def state(x):
    x= str(x)
    if x.split(",")[0].lower().strip() in list(states["name"].str.lower()):
        return "United States"
    elif x.split(",")[-1].lower().strip() in list(states["post_abbr"].str.lower()):
        return "United States"
    elif x.split(",")[0].lower().strip() in list(states["post_abbr"].str.lower()):
        return "United States"
    else:
        return x

def country(x):
    x = str(x)
    for i in iso_countries["Name"]:
        if re.search(fr"{i.lower().strip() }" ,x.lower().strip() ):
            return i.strip() 


In [20]:
users["location"] = users["location"].apply(lambda x: state(x))
users["Country"] = users["location"].apply(lambda x: country(x))

## Assigning Users language 

In [21]:
question_language_dict = dict(zip(postsQ.id, postsQ.language))
postsQ_id = list(postsQ.id)
def assing_lang(x):
    if x in postsQ_id:
        return question_language_dict[x]
    else:
        return "None"
postsA["language"] = postsA["parent_id"].apply(lambda x: assing_lang(x))
postsA = postsA[postsA["language"] != "None"] # filtering out the answers that are not related to questions
print("Start assigning user language")

def assign_user_language(x):
    if len(postsA[postsA["owner_user_id"] == x]["language"]) >0:
        language_A = postsA[postsA["owner_user_id"] == x]["language"].value_counts().index[0]
        language_count_A = postsA[postsA["owner_user_id"] == x]["language"].value_counts()[0]
    else:
        language_A = "None"
        language_count_A = 0
    if len(postsQ[postsQ["owner_user_id"] == x]["language"]) > 0:
        language_Q = postsQ[postsQ["owner_user_id"] == x]["language"].value_counts().index[0]
        language_count_Q = postsQ[postsQ["owner_user_id"] == x]["language"].value_counts()[0]
    else:
        language_Q = "None"
        language_count_Q = 0
    if language_A == language_Q:
        return language_A
    elif language_count_A > language_count_Q:
        return language_A
    else:
        return language_Q
        
    

Start assigning user language


In [22]:
users["language"] = users["id"].apply(lambda x: assign_user_language(x))

In [23]:
len(postsA)

8444

In [24]:
postsA.head()

,id,body,owner_user_id,parent_id,score,creation_date,Hour_of_day,Daily_date,Day_of_week,content,Sentiment,language
62,56909036,"<p><a href=""https://i.stack.imgur.com/1jVhb.jp...",11162165,56905592,25,2019-07-05 20:40:58.003,20,2019-07-05,Friday,stack rel stack alt enter image description st...,5.10,python
98,53592093,<p>You may use <code>json_array_elements</code...,7998591,53591359,2,2018-12-03 10:43:10.447,10,2018-12-03,Monday,may use array access use operator search value...,5.48,sql
154,53255226,<p>When you do this:</p>\n\n<pre><code>int sum...,1876620,53255157,45,2018-11-12 02:18:05.290,2,2018-11-12,Monday,sum map sum every chained method return value ...,5.82,java
386,61712911,<p>You get the common keys from the two lists ...,3888766,61712660,6,2020-05-10 13:34:31.900,13,2020-05-10,Sunday,get common two set intersection method list co...,5.92,python
428,61566726,"<p>Unfortunately, it depends on whether your s...",626853,61566711,17,2020-05-02 21:46:27.820,21,2020-05-02,Saturday,unfortunately whether string least know whethe...,2.76,python


In [25]:
postsQ.head()

,id,title,body,accepted_answer_id,owner_user_id,score,tags,view_count,creation_date,Hour_of_day,Daily_date,Day_of_week,content_title,content_body,Sentiment_title,Sentiment_body,language
96,59684674,Should I add Python's pyc files to .dockerignore?,<p>I've seen several examples of <code>.docker...,59786706,275088,9,python|docker|compilation|docker-build,1084,2020-01-10 15:34:31.800,15,2020-01-10,Friday,add python,p I seen several examples code dockerignore co...,5.90,5.06,python
113,59613434,How to produce map with distinct values from a...,"<p>I have a map <code>Map&lt;K, V&gt;</code> a...",59613610,3764965,13,java|dictionary|java-8|java-stream|collectors,1350,2020-01-06 13:59:06.163,13,2020-01-06,Monday,produce map distinct map use right key,p I map code Map lt K V gt code goal remove du...,6.26,5.06,java
168,57583761,how to show error message in react js when htt...,<p>could you please tell me how to show error ...,57583854,3701974,0,javascript|node.js|reactjs|axios,2017,2019-08-21 02:39:01.900,2,2019-08-21,Wednesday,show error message react request send,p could please tell show error message react j...,6.24,5.06,javascript
172,57302715,how to get input field value on button click i...,<p>Could you please tell me how to get input f...,57303641,3701974,5,javascript|reactjs,3864,2019-08-01 06:06:49.163,6,2019-08-01,Thursday,get input field value button click react,p Could please tell get input field value butt...,5.92,5.06,javascript
183,57242587,Why is the number of local variables used in a...,<p>I have a snippet of simple Java code:</p>\n...,57242675,243990,30,java|jvm|javac|bytecode,3361,2019-07-28 15:58:50.453,15,2019-07-28,Sunday,number local used method economical,p I snippet simple Java code p pre code public...,5.30,5.06,java


In [26]:
users.head()


,id,reputation,display_name,creation_date,location,last_access_date,about_me,views,up_votes,down_votes,access_hour_of_day,Daily_date,Day_of_week,content,Sentiment_aboutme,Sentiment_answer,Sentiment_question,Country,language
0,22656,1206038,Jon Skeet,2008-09-26 12:05:05.150,"Reading, United Kingdom",2020-09-05 17:32:07.613,"<p>\nAuthor of <a href=""https://www.manning.co...",1972774,16633,7090,17,2020-09-05,Saturday,author manning sharp depth fourth edition aid ...,5.98,5.855000,NaN,United Kingdom,java
1,1144035,1006490,Gordon Linoff,2012-01-11 19:53:57.590,United States,2020-09-06 02:21:13.123,"<p>Check out my book <strong><em><a href=""http...",336960,17283,42,2,2020-09-06,Sunday,check book strong click rel data analysis exce...,6.10,5.748571,NaN,United States,sql
2,6309,979770,VonC,2008-09-13 22:22:33.173,France,2020-09-06 05:10:37.823,"<ul>\n<li><p><img src=""http://i.stack.imgur.co...",185652,56359,295,5,2020-09-06,Sunday,stack alt strong system configuration manageme...,5.10,5.383333,NaN,France,python
5,23354,884280,Marc Gravell,2008-09-29 05:46:02.697,"Forest of Dean, United Kingdom",2020-09-05 22:25:38.253,"<p><a href=""http://blog.stackoverflow.com/2010...",291170,25809,1083,22,2020-09-05,Saturday,welcome stack overflow valued stack overflow v...,7.16,5.980000,NaN,United Kingdom,c #
6,115145,869424,CommonsWare,2009-05-31 16:20:08.610,United States,2020-09-05 23:30:27.660,"<p>Mark Murphy, founder of CommonsWare, writes...",244954,11601,7316,23,2020-09-05,Saturday,mark murphy founder lot second generation rece...,5.48,5.070000,NaN,United States,java


In [30]:
users.to_csv("users_processed.csv")
postsQ.to_csv("postsQ_processed.csv")
postsA.to_csv("postsA_processed.csv")